In [4]:
import sys
import os

sys.path.append(os.path.abspath("../.."))

from tiny_graph.graph.executable import Graph
from tiny_graph.models.base import GraphState
from tiny_graph.buffer.factory import History, LastValue
from tiny_graph.checkpoint.storage.local_storage import LocalStorage
from tiny_graph.constants import START, END
import time
from typing import Dict


# Define our state model
class ProcessState(GraphState):
    status: LastValue[str]
    results: History[Dict[str, float]]


# Initialize state and graph with local storage and chain_id
chain_id = "process_workflow_v1"
state = ProcessState(status="", results={})
storage = LocalStorage()
graph = Graph(state=state, checkpoint_storage=storage)


# Define processing nodes
@graph.node()
def initialize_process(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "initializing"}


@graph.node()
def process_data_1(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_1", "results": {"accuracy": 0.85, "step": 1.0}}


@graph.node(interrupt="after")
def process_data_2(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_2", "results": {"accuracy": 0.92, "step": 2.0}}


@graph.node()
def finalize(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "completed"}


# Create the workflow
graph.add_edge(START, "initialize_process")
graph.add_edge("initialize_process", "process_data_1")
graph.add_edge("process_data_1", "process_data_2")
graph.add_edge("process_data_2", "finalize")
graph.add_edge("finalize", END)

# Compile and execute
graph.compile()
graph.visualize()

DEBUG:graphviz.saving:write lines to 'graph'
DEBUG:graphviz.backend.execute:run [PosixPath('dot'), '-Kdot', '-Tpdf', '-O', 'graph']
DEBUG:graphviz.rendering:delete 'graph'
DEBUG:graphviz.backend.viewing:view: ['open', 'graph.pdf']


In [5]:
graph.execute()

DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.RUNNING
DEBUG:tiny_graph.graph.executable:Executing task in node: initialize_process
INFO:tiny_graph.checkpoint.storage.local_storage:Checkpoint '7b7ca2fc-b0e5-4063-95e1-81920f3bdec0' saved in memory.
DEBUG:tiny_graph.graph.executable:State updated after node: initialize_process
DEBUG:tiny_graph.graph.executable:Executing task in node: process_data_1
INFO:tiny_graph.checkpoint.storage.local_storage:Checkpoint 'c8f9c605-bc40-4e92-9df8-972d9d6e6017' saved in memory.
DEBUG:tiny_graph.graph.executable:State updated after node: process_data_1
DEBUG:tiny_graph.graph.executable:Executing task in node: process_data_2
INFO:tiny_graph.checkpoint.storage.local_storage:Checkpoint '1a7ec4da-3d91-4c76-a39c-fa07dda0e00b' saved in memory.
DEBUG:tiny_graph.graph.executable:State updated after node: process_data_2
DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.PAUSE
